In [1]:
uploaded_files = ['Clinical.csv', 'Cytokines.csv']

In [ ]:
st.title("PatStrat: A Framework for Embedding Multi-Modal Similarity Network for Patient Stratification")

In [ ]:
not_null_mask = ~df[col].isnull()
            df.loc[not_null_mask, col] = label_encoder.fit_transform(df.loc[not_null_mask, col])

In [285]:
import pandas as pd
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_regression
warnings.filterwarnings("ignore")
import networkx as nx
import numpy as np 



def custom_mi_reg(a, b):
    a = a.reshape(-1, 1)
    b = b.reshape(-1, 1)
    return  mutual_info_regression(a, b)[0] # should return a float value
    
def preprocess_file(file):
    # Read the CSV file
    df = pd.read_csv(file, index_col=0)
    df = df.transpose()
    
    # Convert categorical columns to numeric using LabelEncoder
    label_encoder = LabelEncoder()
    for col in df:
        if type(df[col][0]) == str:
            df[col] = label_encoder.fit_transform(df[col])
    df = df.transpose().corr(method=custom_mi_reg)
    return df


In [261]:
df = pd.read_csv('data/Clinical.csv', index_col=0)

In [254]:
df = pd.read_csv('data/Cytokines.csv', index_col=0)

In [263]:
def ProcessDF(df):
    for column in df.columns:
        dtype = df[column].dtype
        if dtype == "object":
            label_encoder = LabelEncoder()
            if type(df[column][0]) == str:
                df[column] = label_encoder.fit_transform(df[column])
        elif dtype in ["int64", "float64"]:
            pass
        else:
            raise(ValueError("TypeError"))
        
    df = df.corr(method=custom_mi_reg)
    return df


In [293]:
import plotly.graph_objects as go


In [292]:
from graphviz import Source



In [272]:
G.nodes

NodeView(('MM001', 'MM002', 'MM003', 'MM004', 'MM005', 'MM006', 'MM007', 'MM008', 'MM009', 'MM010', 'MM011', 'MM012', 'MM013', 'MM014', 'MM015', 'MM016', 'MM017', 'MM018', 'MM019', 'MM020', 'MM021', 'MM022', 'MM023', 'MM024', 'MM025', 'MM026', 'MM027', 'MM028', 'MM029', 'MM030', 'MM031', 'MM032', 'MM033', 'MM035', 'MM036', 'MM037', 'MM038', 'MM039', 'MM040', 'MM041', 'MM042', 'MM043', 'MM044', 'MM045', 'MM046', 'MM047', 'MM048', 'MM049', 'MM050', 'MM051', 'MM052', 'MM053', 'MM054', 'MM055', 'MM056', 'MM057', 'MM058', 'MM059', 'MM060', 'MM061', 'MM062', 'MM063', 'MM064', 'MM065', 'MM066', 'MM067', 'MM068', 'MM069', 'MM070', 'MM071', 'MM072', 'MM073', 'MM074', 'MM075', 'MM076', 'MM077', 'MM078', 'MM079', 'MM080', 'MM081', 'MM082', 'MM083', 'MM084', 'MM085', 'MM086', 'MM087', 'MM088', 'MM089', 'MM090', 'MM091', 'MM092', 'MM094', 'MM095', 'MM096', 'MM097', 'MM098', 'MM099', 'MM100', 'MM101', 'MM102', 'MM103', 'MM104', 'MM105', 'MM106', 'MM107', 'MM108', 'MM109', 'MM110', 'MM111', 'MM112', 

In [174]:
# Combine two columns into one
df.index = df['Drugs'] + '_' +  df['Drug class']

# Drop the original columns if needed
df.drop(columns=['Drugs', 'Drug class'], inplace=True)

# Save the modified DataFrame to a new CSV file
df.to_csv('DrugResponse.csv')

In [175]:
df

,MM013,MM014,MM015,MM017,MM018,MM019,MM021,MM023,MM024,MM025,...,MM123,MM125,MM126,MM127,MM129,MM132,MM133,MM135,MM137,MM138
Bendamustine_DNA-alkylating agent,-0.10,-0.05,-0.23,-0.15,-0.08,-0.01,0.14,-0.01,0.01,-0.02,...,-0.15,0.02,-0.04,0.04,-0.28,-0.03,-0.10,0.00,-0.02,-0.05
Bortezomib_Proteasome Inhibitor regime,0.34,0.45,0.65,0.23,0.00,-0.24,0.28,0.18,0.09,0.18,...,0.14,0.27,0.25,0.07,0.48,0.24,0.09,0.15,0.31,0.21
Carfilzomib_Proteasome Inhibitor regime,0.48,0.57,0.66,0.36,0.19,0.41,0.60,0.27,0.34,0.35,...,0.22,0.43,0.21,0.10,0.43,0.25,0.11,0.19,0.32,0.28
Cisplatin_Platinum-containing,-0.17,-0.16,-0.27,-0.22,-0.14,-0.09,0.00,-0.03,-0.08,-0.05,...,0.02,-0.20,-0.08,0.14,-0.14,0.04,0.05,-0.03,0.04,-0.01
Cyclophosphamide_DNA-alkylating agent,-0.15,0.03,-0.05,0.03,-0.08,-0.02,0.01,0.11,0.04,-0.05,...,-0.17,-0.07,-0.06,-0.17,-0.23,-0.03,-0.15,-0.02,-0.03,-0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dexamethasone+Thalidomide_Corticosteroid + IMID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.40,0.08,0.31,0.61,NaN,0.04,0.41,0.13,0.09,0.27
Elotuzumab+Ipilimumab_Immunotherapy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.14,0.20,0.29,-0.01,NaN,0.06,0.12,0.13,-0.06,0.07
Elotuzumab+Nivolumab_Immunotherapy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.24,0.30,0.29,-0.10,NaN,0.07,0.14,0.19,0.00,0.11
Elotuzumab+Pembrolizumab_Immunotherapy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.08,0.28,0.21,-0.10,NaN,0.03,0.16,0.12,-0.01,0.03


In [156]:
df.index.str.split('|')[2]

['IGKV2-28',
 'IGKV2D-29',
 'IGKV2D-30',
 'IGKV2-40',
 'IGKV2D-26',
 'IGKV2-29',
 'IGKV2D-40',
 'IGKV2D-28',
 'IGKV2-30']

In [143]:
new_index = df.index.str.split('|')

In [137]:
new_index = df.index.str.split('|', expand=True).rename(columns={0: 'index_1', 1: 'index_2'})

# Combine new index and repeated values DataFrame
result_df = pd.concat([new_index, repeated_values], axis=1)

TypeError: Index.set_names() got an unexpected keyword argument 'columns'

In [122]:
df = df.transpose()

In [123]:
df.index

Index(['Drugs', 'Drug class', 'MM013', 'MM014', 'MM015', 'MM017', 'MM018',
       'MM019', 'MM021', 'MM023',
       ...
       'MM123', 'MM125', 'MM126', 'MM127', 'MM129', 'MM132', 'MM133', 'MM135',
       'MM137', 'MM138'],
      dtype='object', length=103)

In [114]:
label_encoder = LabelEncoder()
for col in df:
    if type(df[col][0]) == str:
        not_null_mask = ~df[col].isnull()
        df.loc[not_null_mask, col] = label_encoder.fit_transform(df.loc[not_null_mask, col])
        #df[col] = label_encoder.fit_transform(df[col])
df = df.transpose().corr(method=custom_mi_reg)

TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['float', 'str']

In [124]:
df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
Drugs,Bendamustine,Bortezomib,Carfilzomib,Cisplatin,Cyclophosphamide,Cytarabine,Dexamethasone,Etoposide,Ixazomib,Lenalidomide,...,Dexamethasone+Lenalidomide+Elotuzumab,Dexamethasone+Lenalidomide+Pembrolizumab,Dexamethasone+Pomalidomide,Dexamethasone+Pomalidomide+Daratumumab,Dexamethasone+Pomalidomide+Elotuzumab,Dexamethasone+Thalidomide,Elotuzumab+Ipilimumab,Elotuzumab+Nivolumab,Elotuzumab+Pembrolizumab,Ipilimumab+Pembrolizumab
Drug class,DNA-alkylating agent,Proteasome Inhibitor regime,Proteasome Inhibitor regime,Platinum-containing,DNA-alkylating agent,Nucleoitide analog,Corticosteroid,Topoisomerase inhibitor,Proteasome Inhibitor regime,Corticosteroid + IMID,...,Immunotherapy combination,Immunotherapy combination,Corticosteroid + IMID,Immunotherapy combination,Immunotherapy combination,Corticosteroid + IMID,Immunotherapy,Immunotherapy,Immunotherapy,Immunotherapy
MM013,-0.1,0.34,0.48,-0.17,-0.15,0.19,-0.1,0.12,0.22,-0.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MM014,-0.05,0.45,0.57,-0.16,0.03,-0.1,0.45,-0.08,0.23,0.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MM015,-0.23,0.65,0.66,-0.27,-0.05,-0.13,0.19,0.16,0.62,-0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MM132,-0.03,0.24,0.25,0.04,-0.03,0.09,-0.03,0.08,0.09,-0.07,...,0.06,0.04,0.07,0.03,0.06,0.04,0.06,0.07,0.03,0.03
MM133,-0.1,0.09,0.11,0.05,-0.15,-0.12,0.17,0.07,0.14,0.12,...,0.47,0.47,0.33,0.49,0.48,0.41,0.12,0.14,0.16,-0.08
MM135,0.0,0.15,0.19,-0.03,-0.02,0.09,0.13,0.17,0.13,0.03,...,0.16,0.17,0.14,0.22,0.17,0.13,0.13,0.19,0.12,0.06
MM137,-0.02,0.31,0.32,0.04,-0.03,-0.02,0.25,0.16,0.31,-0.02,...,0.13,0.13,0.19,0.14,0.13,0.09,-0.06,0.0,-0.01,-0.02
